<a href="https://www.kaggle.com/code/vladlena321/classification-of-biology-books-log-regr?scriptVersionId=194848163" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/classification-of-biology-textbooks-in-russian/classification_of_biology_textbooks_in_russian.csv


In [2]:
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

import joblib

import torch

from sklearn.preprocessing import StandardScaler

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv("/kaggle/input/classification-of-biology-textbooks-in-russian/classification_of_biology_textbooks_in_russian.csv", sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Название книги  1275 non-null   object
 1   Аннотация       1275 non-null   object
 2   Раздел          1275 non-null   object
 3   Ключевые слова  1275 non-null   object
dtypes: object(4)
memory usage: 40.0+ KB


In [6]:
df.head()

,Название книги,Аннотация,Раздел,Ключевые слова
0,Основы биоинформатики,Учебник охватывает основные концепции и методы...,Биоинформатика и биоинженерия,"биоинформатика, геномный анализ, ДНК, РНК, бел..."
1,Методы секвенирования нового поколения,Книга посвящена методам секвенирования ДНК нов...,Биоинформатика и биоинженерия,"NGS, секвенирование ДНК, Illumina, PacBio, Oxf..."
2,Введение в биоинженерию,Учебное пособие рассматривает основные принцип...,Биоинформатика и биоинженерия,"биоинженерия, биоматериалы, тканевая инженерия..."
3,Анализ биомедицинских данных,Книга охватывает методы анализа биомедицинских...,Биоинформатика и биоинженерия,"биомедицинские данные, машинное обучение в био..."
4,Программирование в биоинформатике,Учебник посвящен программированию на языках Py...,Биоинформатика и биоинженерия,"Python, R, биоинформатика, программирование в ..."


## **Модель на основе TF-IDF + Наивный Байес**

Создаем список стоп-слов, встроенной поддержки стоп-слов для русского языка в scikit-learn нет.

In [7]:
stop_words_russian = [
    'и', 'в', 'на', 'с', 'по', 'для', 'а', 'что', 'не', 'это', 'как', 'из', 'так', 'о', 'то', 'к', 'то'
]

In [8]:
# Создание нового столбца с объединением текста
df['Полный текст'] = df['Название книги'] + ' ' + df['Аннотация'] + ' ' + df['Ключевые слова']

# Определение признаков и целевой переменной
X = df['Полный текст']
y = df['Раздел']

# Векторизация текста
vectorizer = TfidfVectorizer(stop_words=stop_words_russian)
X_transformed = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)
model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8352941176470589
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.86      0.75      0.80         8
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.89      0.89      0.89         9
                  Ботаника высших растений       1.00      0.18      0.31        11
                  Ботаника низших растений       0.92      0.92      0.92        13
Вирусология, микробиология и биотехнология       0.91      0.83      0.87        12
          Генетика и молекулярная биология       0.55      1.00      0.71        21
                   Зоология беспозвоночных       1.00      0.75      0.86        12
                      Зоология позвоночных       0.79      1.00      0.88        15
                               Иммунология       0.90      0.90      0.90        20
            История и м

## TF-IDF + Логистическая Регрессия

In [9]:
model = LogisticRegression(max_iter=5000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.9294117647058824
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.88      0.88      0.88         8
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.89      0.89      0.89         9
                  Ботаника высших растений       1.00      0.73      0.84        11
                  Ботаника низших растений       1.00      0.92      0.96        13
Вирусология, микробиология и биотехнология       0.92      1.00      0.96        12
          Генетика и молекулярная биология       0.83      0.95      0.89        21
                   Зоология беспозвоночных       1.00      1.00      1.00        12
                      Зоология позвоночных       0.93      0.93      0.93        15
                               Иммунология       0.95      0.95      0.95        20
            История и м

Кроссвалидация

In [10]:
# Настройка гиперпараметров
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],  # Добавляем L1 регуляризацию (Lasso)
    'solver': ['liblinear', 'saga'],  # 'liblinear' поддерживает l1 и l2, 'saga' - более быстрая для больших данных
    'max_iter': [1000, 2000, 3000]  # Увеличиваем количество итераций
}

# Кросс-валидация и поиск лучших гиперпараметров
grid_search = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Лучшая модель после настройки гиперпараметров
best_model = grid_search.best_estimator_

# Оценка модели
y_pred = best_model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Кросс-валидация
cv_scores = cross_val_score(best_model, X_transformed, y, cv=5)
print(f'Cross-Validation Accuracy: {cv_scores.mean()}')

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Accuracy: 0.9411764705882353
Classification Report:
                                            precision    recall  f1-score   support

             Биоинформатика и биоинженерия       0.88      0.88      0.88         8
                                 Биофизика       1.00      0.91      0.95        11
                                  Биоэтика       0.89      0.89      0.89         9
                  Ботаника высших растений       1.00      0.73      0.84        11
                  Ботаника низших растений       1.00      0.92      0.96        13
Вирусология, микробиология и биотехнология       0.92      1.00      0.96        12
          Генетика и молекулярная биология       0.83      0.95      0.89        21
                   Зоология беспозвоночных       0.92      1.00      0.96        12
                      Зоология позвоночных       1.00      0.93      0.97        15
                               Иммунология       1.00      0.95      0.97        20
            История и м

Пайплайн

In [11]:
# Предположим, что df — это ваш DataFrame, содержащий столбцы "Название книги", "Аннотация" и "Ключевые слова"
df['Полный текст'] = df['Название книги'] + ' ' + df['Аннотация'] + ' ' + df['Ключевые слова']

# Определите признаки и целевую переменную
X = df['Полный текст']
y = df['Раздел']

# Создание пайплайна
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer(stop_words=stop_words_russian)),
    ('scaler', StandardScaler(with_mean=False)),  # TfidfVectorizer не возвращает среднее значение, поэтому with_mean=False
    ('classifier', LogisticRegression(max_iter=5000))
])

# Определение параметров для настройки гиперпараметров
param_grid = {'classifier__C': [0.1, 1, 10, 100]}

# Кросс-валидация и поиск лучших гиперпараметров
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X, y)

# Лучшая модель после настройки гиперпараметров
best_pipeline = grid_search.best_estimator_

In [12]:
# Сохранение лучшей модели в файл
joblib.dump(best_pipeline, 'best_model.pkl')

['best_model.pkl']

In [13]:
# Загрузка модели из файла
loaded_model = joblib.load('best_model.pkl')

# Пример использования загруженной модели
new_data = ["В монографии изложены результаты изучения распространения апомиктичного способа семенной репродукции у представителей двух экономически важных и эволюционно прогрессивных семейств Asteraceae и Poaceae из флоры европейской части России. Обсуждаются вопросы особенностей проявления апомиксиса у растений, закономерности распределения апомиктичных видов в системе покрытосеменных и распространения их во флоре. Для научных работников, занимающихся проблемами репродуктивной биологии растений и апомиксиса."]
predictions = loaded_model.predict(new_data)

print(predictions)

['Ботаника высших растений']


Пример использования